In [1]:
import pandas as pd
import ydata_profiling as yp
import os
import json
import urllib.request
import webcolors

In [2]:
df = pd.read_csv('../data/full_gen_data.csv')
df.head().T

,0,1,2,3,4
country,Germany,Germany,Germany,Germany,Germany
article,YN8639,YN8639,YN8639,YN8639,YN8639
sales,28,28,28,28,28
regular_price,5.95,5.95,5.95,5.95,5.95
current_price,3.95,3.95,3.95,3.95,3.95
ratio,0.663866,0.663866,0.663866,0.663866,0.663866
retailweek,2016-03-27,2016-03-27,2016-03-27,2016-03-27,2016-03-27
promo1,0,0,0,0,0
promo2,0,0,0,0,0
customer_id,1003.0,1003.0,1003.0,1003.0,1003.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   country         100000 non-null  object 
 1   article         100000 non-null  object 
 2   sales           100000 non-null  int64  
 3   regular_price   100000 non-null  float64
 4   current_price   100000 non-null  float64
 5   ratio           100000 non-null  float64
 6   retailweek      100000 non-null  object 
 7   promo1          100000 non-null  int64  
 8   promo2          100000 non-null  int64  
 9   customer_id     100000 non-null  float64
 10  article.1       100000 non-null  object 
 11  productgroup    100000 non-null  object 
 12  category        100000 non-null  object 
 13  cost            100000 non-null  float64
 14  style           100000 non-null  object 
 15  sizes           100000 non-null  object 
 16  gender          100000 non-null  object 
 17  rgb_r_main_

In [4]:
repeat = False
if not os.path.exists('../reports/first_eda_report.html') or repeat:
    prof = yp.ProfileReport(df)
    prof.to_file('../reports/first_eda_report.html')

* Initial Understanding:
  * No Duplicates or missing values
  * `article`, `article.1` and `customer_id` are all unique identifiers and can be dropped
  * We can derive `discount_pct` as 1 - `ratio` and we need to be aware of multicollinearity
  * We can derive `unit_profit` as `current_price` - `cost`
  * We can then derive `total_profit` as `unit_profit` * `sales`
  * We can also derive `profit_margin` as `unit_profit` / `current_price`
  * `retailweek` needs to be converted to datetime and we can derive `year`, `month`, `week_number` and `day_of_week`
  * We can convert `sizes` to `has_extra_sizes` binary column
  * We can extract `main_color` and `sec_color` from rgb columns

In [5]:
# Copy dataframe
df2 = df.copy()

In [6]:
# Drop IDs columns
df2 = df2.drop(['article', 'customer_id', 'article.1'], axis=1)

In [7]:
# Calculate 'discount_pct'
df2['discount_pct'] = 1 - df2['ratio']

In [8]:
# Calculate `unit_profit`, 'total_profit' and `profit_margin`
df2['unit_profit'] = df2['current_price'] - df2['cost']
df2['total_profit'] = df2['sales'] * df2['unit_profit']
df2['profit_margin'] = df2['unit_profit'] / df2['current_price']

In [9]:
# Handling `retailweek` column
df2['retailweek'] = pd.to_datetime(df2['retailweek'])
df2['year'] = df2['retailweek'].dt.year
df2['month'] = df2['retailweek'].dt.month
df2['week_number'] = df2['retailweek'].dt.isocalendar().week
df2['day_of_week'] = df2['retailweek'].dt.dayofweek
df2 = df2.drop('retailweek', axis=1)

In [10]:
# Handling `sizes`
def has_extra_sizes(sizes):
    if 'xxs' in sizes:
        return 1
    return 0
df2['has_extra_sizes'] = df2['sizes'].apply(has_extra_sizes)
df2 = df2.drop('sizes', axis=1)

In [11]:
# Handling `rgb` columns
url = 'https://raw.githubusercontent.com/bahamas10/css-color-names/master/css-color-names.json'
with urllib.request.urlopen(url) as response:
    css3_names = json.load(response)

# Build name to RGB

name_rgb_dict = {
    name: webcolors.hex_to_rgb(hex_val)
    for name, hex_val in css3_names.items()
}

def closest_color(requested_rgb):
    min_dist = float('inf')
    closest_name = None
    for name, rgb in name_rgb_dict.items():
        dist = sum((comp1 - comp2) ** 2 for comp1, comp2 in zip(rgb, requested_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_name = name
    return closest_name

def rgb_to_color_name(r, g, b):
    try:
        return webcolors.rgb_to_name((r, g, b))
    except ValueError:
        return closest_color((r, g, b))
    

df2['main_color'] = df2.apply(lambda x: rgb_to_color_name(x['rgb_r_main_col'], x['rgb_g_main_col'], x['rgb_b_main_col']), axis=1)
df2['sec_color'] = df2.apply(lambda x: rgb_to_color_name(x['rgb_r_sec_col'], x['rgb_g_sec_col'], x['rgb_b_sec_col']), axis=1)
df2 = df2.drop(['rgb_r_main_col', 'rgb_g_main_col', 'rgb_b_main_col', 'rgb_r_sec_col', 'rgb_g_sec_col', 'rgb_b_sec_col'], axis=1)

In [12]:
df2.head()

,country,sales,regular_price,current_price,ratio,promo1,promo2,productgroup,category,cost,...,unit_profit,total_profit,profit_margin,year,month,week_number,day_of_week,has_extra_sizes,main_color,sec_color
0,Germany,28,5.95,3.95,0.663866,0,0,SHOES,TRAINING,13.29,...,-9.34,-261.52,-2.364557,2016,3,12,6,1,chocolate,lavender
1,Germany,28,5.95,3.95,0.663866,0,0,SHORTS,TRAINING,2.29,...,1.66,46.48,0.420253,2016,3,12,6,1,darkkhaki,lavender
2,Germany,28,5.95,3.95,0.663866,0,0,HARDWARE ACCESSORIES,GOLF,1.70,...,2.25,63.00,0.569620,2016,3,12,6,1,goldenrod,lavender
3,Germany,28,5.95,3.95,0.663866,0,0,SHOES,RUNNING,9.00,...,-5.05,-141.40,-1.278481,2016,3,12,6,1,rosybrown,lightblue
4,Germany,28,5.95,3.95,0.663866,0,0,SHOES,RELAX CASUAL,9.60,...,-5.65,-158.20,-1.430380,2016,3,12,6,1,blueviolet,lightblue


In [13]:
# Rearrange columns
df2 = df2[['country', 'productgroup', 'category', 'style', 'gender', 'main_color', 'sec_color', 'has_extra_sizes',
        'year', 'month', 'week_number', 'day_of_week', 'regular_price', 'current_price', 'ratio', 'discount_pct', 'cost', 'sales',
        'unit_profit',	'total_profit',	'profit_margin', 'promo1', 'promo2', 'label']]

df2.head()

,country,productgroup,category,style,gender,main_color,sec_color,has_extra_sizes,year,month,...,ratio,discount_pct,cost,sales,unit_profit,total_profit,profit_margin,promo1,promo2,label
0,Germany,SHOES,TRAINING,slim,women,chocolate,lavender,1,2016,3,...,0.663866,0.336134,13.29,28,-9.34,-261.52,-2.364557,0,0,0
1,Germany,SHORTS,TRAINING,regular,women,darkkhaki,lavender,1,2016,3,...,0.663866,0.336134,2.29,28,1.66,46.48,0.420253,0,0,0
2,Germany,HARDWARE ACCESSORIES,GOLF,regular,women,goldenrod,lavender,1,2016,3,...,0.663866,0.336134,1.70,28,2.25,63.00,0.569620,0,0,0
3,Germany,SHOES,RUNNING,regular,kids,rosybrown,lightblue,1,2016,3,...,0.663866,0.336134,9.00,28,-5.05,-141.40,-1.278481,0,0,0
4,Germany,SHOES,RELAX CASUAL,regular,women,blueviolet,lightblue,1,2016,3,...,0.663866,0.336134,9.60,28,-5.65,-158.20,-1.430380,0,0,0


In [14]:
df2.describe().round(2)

,has_extra_sizes,year,month,week_number,day_of_week,regular_price,current_price,ratio,discount_pct,cost,sales,unit_profit,total_profit,profit_margin,promo1,promo2,label
count,100000.0,100000.00,100000.00,100000.0,100000.0,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00
mean,0.9,2015.70,5.91,24.34,6.0,52.39,28.29,0.55,0.45,6.52,56.78,21.77,997.10,0.53,0.06,0.00,0.14
std,0.3,0.72,3.53,15.61,0.0,35.27,22.58,0.19,0.19,3.91,87.93,22.92,2258.27,0.70,0.24,0.07,0.35
min,0.0,2014.00,1.00,1.0,6.0,3.95,1.95,0.30,0.00,1.29,1.00,-11.34,-9026.64,-5.82,0.00,0.00,0.00
25%,1.0,2015.00,3.00,10.75,6.0,25.95,11.95,0.35,0.30,2.29,10.00,5.66,73.30,0.45,0.00,0.00,0.00
50%,1.0,2016.00,5.00,22.0,6.0,40.95,20.95,0.53,0.47,6.95,26.00,15.35,316.54,0.75,0.00,0.00,0.00
75%,1.0,2016.00,9.00,38.0,6.0,79.95,37.95,0.70,0.65,9.60,64.00,31.66,997.54,0.89,0.00,0.00,0.00
max,1.0,2017.00,12.00,53.0,6.0,197.95,195.95,1.00,0.70,13.29,898.00,194.66,57743.06,0.99,1.00,1.00,1.00


In [15]:
# Remove `day_of_week` as it has constant value (6 : Sunday)
df2 = df2.drop('day_of_week', axis=1)

In [16]:
# Handling Data Types
for col in ['year', 'month', 'week_number', 'promo1', 'promo2', 'label', 'has_extra_sizes', 'country',
             'productgroup', 'category', 'style', 'gender', 'main_color', 'sec_color']:
    df2[col] = df2[col].astype('category')

In [17]:
df2.describe(include='category')

,country,productgroup,category,style,gender,main_color,sec_color,has_extra_sizes,year,month,week_number,promo1,promo2,label
count,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000
unique,3,4,6,3,4,10,3,2,4,12,53,2,2,2
top,Germany,SHOES,TRAINING,regular,women,blueviolet,rosybrown,1,2015,1,3,0,0,0
freq,49400,60000,30000,50000,70000,10000,40000,90000,42790,11730,2770,93810,99510,86072


In [18]:
df2.describe().round(2)

,regular_price,current_price,ratio,discount_pct,cost,sales,unit_profit,total_profit,profit_margin
count,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00
mean,52.39,28.29,0.55,0.45,6.52,56.78,21.77,997.10,0.53
std,35.27,22.58,0.19,0.19,3.91,87.93,22.92,2258.27,0.70
min,3.95,1.95,0.30,0.00,1.29,1.00,-11.34,-9026.64,-5.82
25%,25.95,11.95,0.35,0.30,2.29,10.00,5.66,73.30,0.45
50%,40.95,20.95,0.53,0.47,6.95,26.00,15.35,316.54,0.75
75%,79.95,37.95,0.70,0.65,9.60,64.00,31.66,997.54,0.89
max,197.95,195.95,1.00,0.70,13.29,898.00,194.66,57743.06,0.99


In [19]:
df2.columns

Index(['country', 'productgroup', 'category', 'style', 'gender', 'main_color',
       'sec_color', 'has_extra_sizes', 'year', 'month', 'week_number',
       'regular_price', 'current_price', 'ratio', 'discount_pct', 'cost',
       'sales', 'unit_profit', 'total_profit', 'profit_margin', 'promo1',
       'promo2', 'label'],
      dtype='object')

* Now, We have Our Attributes:
    * 🌍 Customer/Market Context: `country`
    * 🧩 Product Attributes: `productgroup`, `category`, `style`, `gender`, `has_extra_sizes`, `main_color`, `sec_color`
    * 📆 Time Context: `year`, `month`, `week_number`
    * 💰 Pricing & Cost: `regular_price`, `current_price`, `ratio`, `discount_pct`, `cost`
    * 📈 Sales & Profitability: `sales`, `unit_profit`, `total_profit`, `profit_margin`
    * 📢 Marketing Promotions: `promo1`, `promo2`
    * 🎯 Target: `label`

In [20]:
df2.to_csv('../data/data_feature_engineered.csv', index=False)